In [17]:
import os
import numpy as np
import shutil
import random
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import ModelCheckpoint

### split train test data

In [3]:
files_name=os.listdir('asl_dataset')

In [21]:
def test_images(list_of_filesname):
    for file_name in list_of_filesname:
        os.mkdir(f'asl_dataset_test/{file_name}')
        path_of_images_test=random.sample(os.listdir(f'asl_dataset/{file_name}'), 17)
        for image_path in path_of_images_test:
            shutil.move(f'asl_dataset/{file_name}/{image_path}', f'asl_dataset_test/{file_name}/{image_path}')
        
test_images(files_name)

### neural

In [23]:
image_generator_train = ImageDataGenerator(
    rescale=1./255.,
    shear_range=0.1,
    zoom_range=0.3,
)

image_generator_test = ImageDataGenerator(
    rescale=1./255.
)

train_data= image_generator_train.flow_from_directory(
    'asl_dataset',
    target_size=(500, 500),
    class_mode='categorical',
    batch_size=10
)

test_data= image_generator_test.flow_from_directory(
    'asl_dataset_test',
    target_size=(500, 500),
    class_mode='categorical',
    batch_size=10
)

Found 1886 images belonging to 36 classes.
Found 612 images belonging to 36 classes.


In [29]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(500, 500, 3)))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(36, activation='softmax'))

In [31]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [32]:
es = EarlyStopping(monitor='val_loss', mode='min', patience=5)
rlrop = ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.2, min_lr=0.001)

In [33]:
mch = ModelCheckpoint('chestxray.h5', monitor='val_loss', mode='min', save_best_only=True)

In [34]:
history = model.fit(train_data, steps_per_epoch=len(train_data), epochs=5, validation_data=test_data, validation_steps=len(test_data), callbacks=[es, rlrop, mch])

Epoch 1/5
189/189 [==============================] - 6151s 33s/step - loss: 3.1622 - categorical_accuracy: 0.2603 - val_loss: 1.1216 - val_categorical_accuracy: 0.6650 - lr: 0.0010
Epoch 2/5
189/189 [==============================] - 6125s 32s/step - loss: 0.9895 - categorical_accuracy: 0.6967 - val_loss: 0.7032 - val_categorical_accuracy: 0.7761 - lr: 0.0010
Epoch 3/5
189/189 [==============================] - 5799s 31s/step - loss: 0.5973 - categorical_accuracy: 0.8134 - val_loss: 0.4003 - val_categorical_accuracy: 0.8578 - lr: 0.0010
Epoch 4/5
189/189 [==============================] - 5869s 31s/step - loss: 0.4436 - categorical_accuracy: 0.8515 - val_loss: 0.3628 - val_categorical_accuracy: 0.8758 - lr: 0.0010
Epoch 5/5
189/189 [==============================] - 16024s 85s/step - loss: 0.3799 - categorical_accuracy: 0.8738 - val_loss: 0.3595 - val_categorical_accuracy: 0.8856 - lr: 0.0010
